# Assignment 2: Support Vector Machines

Jingran Lyu a1832252

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn.metrics import accuracy_score

In [2]:
# Data set
train = pd.read_csv('/Users/shawnl/Downloads/train.csv', header=None)
test = pd.read_csv('/Users/shawnl/Downloads/test.csv', header=None)
# train.head()
# test.head()
x_train, x_valid = train.iloc[:4000, 1:], train.iloc[4000:, 1:]
y_train, y_valid = train.iloc[:4000, 0], train.iloc[4000:, 0]
x_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]
y_train, y_valid, y_test = np.where(y_train == 0, -1, y_train), \
                            np.where(y_valid == 0, -1, y_valid), \
                            np.where(y_test == 0, -1, y_test)

The primal form of soft-margin Linear Support Vector Machine

In [3]:
# Primal form of soft-margin SVM trainning
def svm_train_primal (data_train ,label_train ,regularisation_para_C):
    m, n = data_train.shape
    p_w, p_b, p_xi = np.asarray(np.diag(np.ones(n))), np.zeros([1,1]), np.zeros([m, n])
    p_w_s, p_b_s, p_xi_s= np.zeros([1, n]), np.zeros([1, m]), np.zeros((m, m))

    P = cvxopt_matrix(np.concatenate((np.concatenate((p_w, p_w_s.T, p_xi.T), axis = 1), 
                       np.concatenate((p_xi, p_b_s.T, p_xi_s), axis = 1),
                       np.concatenate((p_w_s, p_b, p_b_s), axis = 1)), axis = 0))
    
    q = cvxopt_matrix(np.concatenate((p_w_s, p_b, np.ones([1, m]) * (regularisation_para_C / m)), axis = 1).T)
    
    g_1 = np.dot(np.diag(label_train), data_train) * (-1)
    g_2 = (np.matrix(label_train) * (-1)).T
    g_3 = np.diag(np.ones([m]) * (-1))
    G = cvxopt_matrix(np.concatenate((np.concatenate((g_1, g_2, g_3), axis = 1),
                      np.concatenate((np.zeros([m, n]), np.zeros([m, 1]), g_3), axis = 1)), axis = 0))
    
    h = cvxopt_matrix(np.concatenate((np.ones([m, 1]) * -1, np.zeros([m, 1])), axis = 0))
    primal_model = np.array(cvxopt_solvers.qp(P,q,G,h)['x']).flatten()
    return primal_model

In [4]:
svm_model_primal = svm_train_primal(x_train, y_train, 100)

     pcost       dcost       gap    pres   dres
 0:  7.1592e+00  6.0250e+02  4e+04  3e+00  2e+04
 1:  2.1840e+02 -8.6176e+02  1e+03  7e-02  4e+02
 2:  1.3725e+02 -9.0183e+01  2e+02  1e-02  5e+01
 3:  4.8478e+01 -1.5479e+01  6e+01  3e-03  1e+01
 4:  2.4467e+01 -2.7209e+00  3e+01  1e-03  6e+00
 5:  1.1255e+01  3.8462e+00  7e+00  3e-04  1e+00
 6:  7.9712e+00  5.4852e+00  2e+00  6e-05  3e-01
 7:  6.9506e+00  5.9990e+00  1e+00  2e-05  9e-02
 8:  6.5698e+00  6.1903e+00  4e-01  2e-06  1e-02
 9:  6.4094e+00  6.2999e+00  1e-01  5e-07  3e-03
10:  6.3622e+00  6.3341e+00  3e-02  9e-08  5e-04
11:  6.3477e+00  6.3454e+00  2e-03  4e-09  2e-05
12:  6.3465e+00  6.3464e+00  2e-04  1e-10  7e-07
13:  6.3464e+00  6.3464e+00  4e-06  3e-12  1e-08
Optimal solution found.


In [5]:
# Primal form testing
def svm_predict_primal (data_test , label_test , model):
    m, n = data_test.shape
    accuracy = accuracy_score(label_test, np.sign(np.dot(data_test, model[:n]) + model[n]))
    return accuracy
# print(svm_predict_primal(x_test, y_test, svm_model_primal))

In [6]:
# Quick check w and b
def quick_check_primal(data_train, label_train, model):
    w_primal, b_primal = np.sum(model[:data_train.shape[1]]), model[data_train.shape[1]]
    print(w_primal , b_primal)
    xi = model[data_train.shape[1]+1:]
    support_vectors = np.where(xi < 1e-6, True, False)
    xi = xi[support_vectors]
    # print(xi.shape)
quick_check_primal(x_train, y_train, svm_model_primal)

-0.14513522739480422 1.7798092086721722


The dual form of soft-margin Linear Support Vector Machine

In [7]:
# Dual form of soft-margin SVM training
def svm_train_dual (data_train ,label_train ,regularisation_para_C):
    m, n = data_train.shape
    p_y = np.diag(label_train).T
    p_yx = np.dot(p_y, data_train)
    P = cvxopt_matrix(np.dot(p_yx, p_yx.T))
    q = cvxopt_matrix((np.ones([1, m]) * -1).T)
    G = cvxopt_matrix(np.concatenate((np.diag(np.ones([m])), np.diag(np.ones([m])) * -1), axis = 0))
    h = cvxopt_matrix(np.concatenate((np.ones([m, 1]) * (regularisation_para_C / m), np.zeros([m, 1])), axis = 0))
    A = cvxopt_matrix(np.matrix(y_train))
    b = cvxopt_matrix(np.zeros([1]))
    dual_model = np.array(cvxopt_solvers.qp(P, q, G, h, A, b)["x"]).flatten()
    return dual_model

In [8]:
svm_model_dual = svm_train_dual(x_train, y_train, 100)

     pcost       dcost       gap    pres   dres
 0: -3.8416e+02 -2.2800e+02  4e+04  1e+02  5e-13
 1: -2.2204e+01 -2.2367e+02  1e+03  3e+00  5e-13
 2: -1.2704e+01 -1.3798e+02  2e+02  4e-01  7e-14
 3: -8.2452e+00 -4.8671e+01  6e+01  1e-01  3e-14
 4: -6.5928e+00 -2.4543e+01  3e+01  4e-02  2e-14
 5: -5.8533e+00 -1.1273e+01  7e+00  1e-02  2e-14
 6: -5.9382e+00 -7.9755e+00  2e+00  2e-03  2e-14
 7: -6.1146e+00 -6.9517e+00  1e+00  7e-04  2e-14
 8: -6.2067e+00 -6.5700e+00  4e-01  1e-04  2e-14
 9: -6.3032e+00 -6.4094e+00  1e-01  2e-05  2e-14
10: -6.3347e+00 -6.3622e+00  3e-02  4e-06  2e-14
11: -6.3454e+00 -6.3477e+00  2e-03  1e-07  2e-14
12: -6.3464e+00 -6.3465e+00  2e-04  5e-09  2e-14
13: -6.3464e+00 -6.3464e+00  4e-06  1e-10  2e-14
Optimal solution found.


In [9]:
# Dual form testing
def svm_predict_dual (data_test , label_test , model):
    dual_model = obtain_primal_from_dual(x_train, y_train, model)
    accuracy = accuracy_score(label_test, np.sign(np.dot(data_test, dual_model[:-1]) + dual_model[-1]))
    return accuracy
# svm_predict_dual(x_train, y_train, svm_model_dual)

In [10]:
# Quick check alpha
def quick_check_dual(model):
    a_dual = np.sum(model)
    print(a_dual)
quick_check_dual(svm_model_dual)

7.281637057117681


In [108]:
# Obtain the primal problem solution w,b from its dual solution alpha
def obtain_primal_from_dual(data_train, label_train, model_primal, model_dual):
    # model = model.T
    # model = model[support_vectors]
    # model = model.T
    w_star = np.dot(np.matrix(model_dual), np.diag(label_train).T)
    w_star = np.dot(w_star, data_train)
    print(w_star.shape)
    w_star_sum = np.sum(w_star)

    xi = model_primal[data_train.shape[1]+1:]
    support_vectors = np.where(xi < 1e-4, True, False)
    model_dual = model_dual[support_vectors]
    data_train = data_train[support_vectors]
    label_train = label_train[support_vectors]
    support_vectors = np.where(model_dual < 1e-4, False, True)
    # model_dual = model_dual[support_vectors]
    # print(model_dual.shape)
    data_train = data_train[support_vectors]
    label_train = label_train[support_vectors]
    b_star = np.dot(w_star, data_train.T)
    b_star = np.matrix(label_train) - b_star
    b_star_sum = b_star.mean()
    dual_model = np.concatenate((w_star.T,np.matrix(b_star_sum)), axis=0)
    print(w_star_sum, b_star_sum)
    return dual_model
obtain_primal_from_dual(x_train, y_train, svm_model_primal, svm_model_dual)

(1, 200)
-0.1451352273952501 1.7798092842125055


matrix([[-2.62255924e-02],
        [-1.14172087e-01],
        [ 4.18625306e-02],
        [-5.22990713e-02],
        [ 7.55230745e-02],
        [-7.49278322e-03],
        [ 2.79583447e-02],
        [-1.01830347e-02],
        [ 5.69949876e-03],
        [ 9.35643587e-04],
        [ 9.63096129e-02],
        [ 6.44609832e-02],
        [ 6.85387186e-02],
        [ 6.24582054e-02],
        [ 1.82642690e-02],
        [-2.39426375e-02],
        [-5.77822608e-02],
        [-4.51056352e-02],
        [-5.51932768e-02],
        [-2.57671265e-02],
        [-2.12456034e-03],
        [ 4.38299699e-02],
        [-8.61260521e-03],
        [-4.62287667e-02],
        [-2.00099113e-02],
        [-1.86215919e-02],
        [-3.54752780e-03],
        [ 9.43200833e-03],
        [-7.37746304e-02],
        [ 1.15434720e-01],
        [-2.37134176e-02],
        [-1.37868546e-02],
        [ 6.08666321e-02],
        [-2.12018774e-02],
        [ 8.70499830e-02],
        [-3.83765354e-02],
        [ 2.25724478e-02],
 

In [109]:
# Find the support vectors from the primal problem solutions
def obtain_support_vectors_primal(data_train, label_train, model):
    w_primal, b_primal, xi = model[:data_train.shape[1]], model[data_train.shape[1]], model[data_train.shape[1]+1:]
    support_vectors = np.where(xi < 1e-4, True, False)

    for i in range(len(support_vectors)):
        if label_train[i] * (np.dot(np.matrix(w_primal), np.matrix(data_train.loc[i].values).T)  + b_primal) -1 > 1e-4:
            support_vectors[i] = False
    data_train = data_train[support_vectors]
    label_train = label_train[support_vectors]
    # print(support_vectors[support_vectors].shape)
    print(data_train, label_train)
obtain_support_vectors_primal(x_train, y_train, svm_model_primal)

       1     2     3     4     5     6     7     8     9     10   ...   191  \
16    1.05 -1.79  0.90  0.68  0.54 -1.93 -0.71 -0.18  1.40  0.73  ...  1.18   
54    0.29 -2.31 -0.40 -0.73  0.11  0.39 -0.58 -0.16  2.31  2.19  ...  0.62   
88    1.65  0.32 -0.72  0.95  1.07  2.57  1.22  1.19  0.54  0.05  ... -1.69   
130  -0.59  0.41  0.13  2.90  0.43 -0.27  0.66 -0.59 -1.10  0.51  ... -0.74   
145  -0.83  1.32 -0.55  1.65  0.46  0.30  1.28  0.91  1.93  1.69  ... -0.47   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
3893 -0.44  1.15  0.95  1.38 -0.04  0.15  0.27  2.19  0.37  0.02  ...  1.13   
3916  0.15 -0.74  1.07 -0.64 -0.74 -0.82 -0.01  0.06 -1.48 -1.37  ...  0.66   
3942  0.33 -1.00 -0.00 -1.26  0.65  1.27  0.95 -0.44 -1.22 -0.56  ...  0.04   
3953  0.16 -1.60  1.34  1.34 -0.10  0.22  0.99  0.81  2.00 -1.54  ...  2.20   
3969  0.79  1.01  1.20 -0.78  1.69  1.52 -0.30 -1.12  0.15 -2.31  ... -0.39   

       192   193   194   195   196   197   198   19

In [110]:
# Find the support vectors from the dual problem solutions
def obtain_support_vectors_dual(data_train, label_train, model_primal, model_dual):
    xi = model_primal[data_train.shape[1]+1:]
    support_vectors = np.where(xi < 1e-4, True, False)
    model_dual = model_dual[support_vectors]
    data_train = data_train[support_vectors]
    label_train = label_train[support_vectors]
    support_vectors = np.where(model_dual < 1e-4, False, True)
    data_train = data_train[support_vectors]
    label_train = label_train[support_vectors]
    print(data_train, label_train)
obtain_support_vectors_dual(x_train, y_train, svm_model_primal, svm_model_dual)

       1     2     3     4     5     6     7     8     9     10   ...   191  \
16    1.05 -1.79  0.90  0.68  0.54 -1.93 -0.71 -0.18  1.40  0.73  ...  1.18   
54    0.29 -2.31 -0.40 -0.73  0.11  0.39 -0.58 -0.16  2.31  2.19  ...  0.62   
88    1.65  0.32 -0.72  0.95  1.07  2.57  1.22  1.19  0.54  0.05  ... -1.69   
130  -0.59  0.41  0.13  2.90  0.43 -0.27  0.66 -0.59 -1.10  0.51  ... -0.74   
145  -0.83  1.32 -0.55  1.65  0.46  0.30  1.28  0.91  1.93  1.69  ... -0.47   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
3893 -0.44  1.15  0.95  1.38 -0.04  0.15  0.27  2.19  0.37  0.02  ...  1.13   
3916  0.15 -0.74  1.07 -0.64 -0.74 -0.82 -0.01  0.06 -1.48 -1.37  ...  0.66   
3942  0.33 -1.00 -0.00 -1.26  0.65  1.27  0.95 -0.44 -1.22 -0.56  ...  0.04   
3953  0.16 -1.60  1.34  1.34 -0.10  0.22  0.99  0.81  2.00 -1.54  ...  2.20   
3969  0.79  1.01  1.20 -0.78  1.69  1.52 -0.30 -1.12  0.15 -2.31  ... -0.39   

       192   193   194   195   196   197   198   19

Search optimal C

In [14]:
# Search optimal C in primal
para_c = list(np.logspace(-5, 5, 11, base=4))
optimal_c = 0
test_accuracy = 0
for c in para_c:
    model_primal_c = svm_train_primal(x_train , y_train , c)
    test_accuracy_primal = svm_predict_primal(x_valid , y_valid , model_primal_c)
    if test_accuracy < test_accuracy_primal:
        optimal_c = c
        test_accuracy = test_accuracy_primal
    print('The training accuracy for C = ', c, ' is: ', test_accuracy_primal)
print("The optimal C found in the validation set is: ", optimal_c)

     pcost       dcost       gap    pres   dres
 0:  5.1377e-02  3.7479e+02  4e+04  3e+00  3e+04
 1:  5.3454e-02 -3.7472e+02  4e+02  3e-02  3e+02
 2:  5.0804e-02 -6.1234e+00  6e+00  5e-04  4e+00
 3:  1.7420e-02 -6.8341e-01  7e-01  6e-05  5e-01
 4:  2.9628e-03 -4.7427e-02  5e-02  4e-06  3e-02
 5:  2.6602e-03  7.5954e-04  2e-03  7e-16  1e-17
 6:  1.3649e-03  8.3815e-04  5e-04  4e-16  3e-18
 7:  1.0394e-03  9.2916e-04  1e-04  3e-16  2e-18
 8:  9.6442e-04  9.5172e-04  1e-05  3e-16  3e-17
 9:  9.5308e-04  9.5197e-04  1e-06  3e-16  8e-17
10:  9.5290e-04  9.5196e-04  9e-07  5e-16  4e-17
11:  9.5284e-04  9.5197e-04  9e-07  4e-16  3e-17
12:  9.5273e-04  9.5199e-04  7e-07  4e-16  3e-17
13:  9.5260e-04  9.5204e-04  6e-07  3e-16  2e-17
14:  9.5248e-04  9.5208e-04  4e-07  3e-16  2e-17
15:  9.5240e-04  9.5212e-04  3e-07  3e-16  1e-17
16:  9.5234e-04  9.5215e-04  2e-07  3e-16  9e-18
17:  9.5229e-04  9.5218e-04  1e-07  3e-16  1e-17
18:  9.5226e-04  9.5219e-04  7e-08  3e-16  2e-17
Optimal solution foun

In [15]:
# Search C in dual
# para_c = list(range(1,101,5))
# for c in para_c:
#     model_dual_c = svm_train_dual(x_train , y_train , c)
#     test_accuracy_dual = svm_predict_dual(x_valid , y_valid , model_dual_c)
#     print('The training accuracy for C = ', c, ' is: ', test_accuracy_dual)

Scikit-learn SVM

In [111]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# sklearn with linear SVM
def sklearn_svm(data_train, label_train, regularisation_para_C, data_test, label_test):
    clf = make_pipeline(StandardScaler(), LinearSVC(dual=False, tol=1e-5, C=regularisation_para_C, random_state=0))
    clf.fit(data_train, label_train)
    accuracy = accuracy_score(label_test, clf.predict(data_test))
    return accuracy

skl_accuracy = sklearn_svm(x_train, y_train, optimal_c, x_valid, y_valid)
print('The training accuracy for optimal C = ', optimal_c, ' is: ', skl_accuracy)

The training accuracy for optimal C =  4.0  is:  0.964
